# Import


In [1]:
import time
import json
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
from sklearn.metrics import confusion_matrix

# Code

### 4 sizes of batch size to compare the results

In [2]:
batch_sizes = [32, 64, 128, 256]

### Preprocessing

In [3]:
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255.0, label

### Splitting dataset into train and test sets

In [4]:
(ds_train_orig, ds_val_orig), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    as_supervised=True,
    with_info=True,
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.3WTMTP_3.0.1/mnist-train.tfrecord*...:   0%|          | 0…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.3WTMTP_3.0.1/mnist-test.tfrecord*...:   0%|          | 0/…

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


### Training

In [5]:
all_train_losses = {}
all_val_losses = {}
all_train_accuracies = {}
all_val_accuracies = {}
training_times = {}
confusion_matrices = {}

In [6]:
for batch_size in batch_sizes:
    print(f"\nTraining with batch size: {batch_size}")

    ds_train = ds_train_orig.map(normalize_img).shuffle(1024).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    ds_val = ds_val_orig.map(normalize_img).batch(batch_size).prefetch(tf.data.AUTOTUNE)

    # Model
    model = models.Sequential([
        layers.Input(shape=(28, 28, 1)),
        layers.Conv2D(8, kernel_size=2, strides=1, padding="same", activation='relu'),
        layers.MaxPooling2D(pool_size=2, strides=2),
        layers.Dropout(0.3),
        layers.Flatten(),
        layers.Dense(10)
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

    early_stop = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

    # Time
    start_time = time.time()
    history = model.fit(
        ds_train,
        epochs=50,
        validation_data=ds_val,
        callbacks=[early_stop],
        verbose=2
    )
    training_time = time.time() - start_time

    # Evaulation
    val_loss, val_acc = model.evaluate(ds_val, verbose=0)

    train_loss = history.history['loss']
    val_loss_hist = history.history['val_loss']
    train_acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    epochs_done = len(train_loss)

    # Prediction and Confusion Matrix
    y_true = []
    y_pred = []

    for images, labels in ds_val:
        logits = model.predict(images, verbose=0)
        preds = np.argmax(logits, axis=1)
        y_true.extend(labels.numpy())
        y_pred.extend(preds)

    cm = confusion_matrix(y_true, y_pred)

    # Saving resulst to JSON file
    results = {
        "train_loss": train_loss,
        "val_loss": val_loss_hist,
        "train_accuracy": train_acc,
        "val_accuracy": val_acc,
        "training_time_seconds": training_time,
        "confusion_matrix": cm.tolist(),
        "epochs_done": epochs_done
    }

    filename = f"/content/[{batch_size}]tf_cnn_results.json"
    with open(filename, "w") as f:
        json.dump(results, f)

    print(f"Results saved: {filename}")



Training with batch size: 32
Epoch 1/50
1875/1875 - 12s - 7ms/step - accuracy: 0.9184 - loss: 0.2706 - val_accuracy: 0.9660 - val_loss: 0.1232
Epoch 2/50
1875/1875 - 6s - 3ms/step - accuracy: 0.9525 - loss: 0.1559 - val_accuracy: 0.9699 - val_loss: 0.1010
Epoch 3/50
1875/1875 - 7s - 4ms/step - accuracy: 0.9584 - loss: 0.1313 - val_accuracy: 0.9745 - val_loss: 0.0861
Epoch 4/50
1875/1875 - 6s - 3ms/step - accuracy: 0.9629 - loss: 0.1191 - val_accuracy: 0.9752 - val_loss: 0.0813
Epoch 5/50
1875/1875 - 7s - 4ms/step - accuracy: 0.9657 - loss: 0.1099 - val_accuracy: 0.9765 - val_loss: 0.0749
Epoch 6/50
1875/1875 - 10s - 6ms/step - accuracy: 0.9676 - loss: 0.1043 - val_accuracy: 0.9759 - val_loss: 0.0737
Epoch 7/50
1875/1875 - 6s - 3ms/step - accuracy: 0.9692 - loss: 0.0994 - val_accuracy: 0.9759 - val_loss: 0.0747
Epoch 8/50
1875/1875 - 10s - 5ms/step - accuracy: 0.9692 - loss: 0.0970 - val_accuracy: 0.9777 - val_loss: 0.0692
Epoch 9/50
1875/1875 - 11s - 6ms/step - accuracy: 0.9708 - loss